# 프로젝트명 : Movielens 영화 추천 실습

이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1. 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [61]:
import os
import pandas as pd
rating_file_path = os.getenv('HOME') + '/aiffel/exploration/E09/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [62]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating'] >= 3]
filtered_data_size = len(ratings)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

original_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [63]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [64]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [65]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/exploration/E09/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2. 데이터 분석

__ratings에 있는 유니크한 영화 개수__

In [66]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

__rating에 있는 유니크한 사용자 수__

In [67]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

__가장 인기 있는 영화 30개(인기순)__

In [68]:
# 앞에서 불러온 2개의 데이터를 1개로 합쳐준다.
# 2개 데이터에서 공통으로 가지고 있는 movie_id 기준으로 데이터를 merge 한다.
movie_df = pd.merge(ratings, movies, on='movie_id')
movie_df.shape

(836478, 6)

In [69]:
# 기존 ratings 데이터에 title, genre 컬럼이 추가된 movie_df가 만들어졌다.
movie_df.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [70]:
# 가장 인기있는 영화 30선 출력
movie_df.groupby('title')['user_id'].count().sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

__기타__

In [71]:
# 유저별 몇개의 영화에 대해 평가했는지에 대한 통계
user_count = movie_df.groupby('title')['user_id'].count()
user_count.describe()

count    3628.000000
mean      230.561742
std       355.596393
min         1.000000
25%        23.000000
50%        87.000000
75%       285.000000
max      3211.000000
Name: user_id, dtype: float64

In [72]:
# 필요없는 컬럼 제외
movie_df = movie_df.drop(columns=['movie_id', 'timestamp', 'genre'])
movie_df.head()

,user_id,count,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,2,5,One Flew Over the Cuckoo's Nest (1975)
2,12,4,One Flew Over the Cuckoo's Nest (1975)
3,15,4,One Flew Over the Cuckoo's Nest (1975)
4,17,5,One Flew Over the Cuckoo's Nest (1975)


## 3. 내가 선호하는 영화 5가지를 추가하기

데이터 인덱싱을 통해 내가 선호하는 영화 제목을 찾아본다.  
단, 2000년대 이후 영화는 검색이 안되는 것 같다.

In [73]:
movie_df[movie_df['title'].str.lower().str.contains('home alone 3')].head(1)

,user_id,count,title
771772,75,5,Home Alone 3 (1997)


In [74]:
# 내가 선호하는 영화 5개
my_favorite = [
    'Terminator 2: Judgment Day (1991)' , 
    'Parent Trap, The (1998)' ,
    'Home Alone 3 (1997)', 
    'Notting Hill (1999)',
    'Shawshank Redemption, The (1994)'
]

# 유저id, 영화제목, 평점 추가
my_rating = pd.DataFrame({'user_id': [9999]*5, 'title': my_favorite, 'count': [5]*5})

In [75]:
if not movie_df.isin({'user_id':[9999]})['user_id'].any():  # user_id에 '9999'라는 데이터가 없다면
    movie_df = movie_df.append(my_rating)                   # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

movie_df.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,count,title
836473,5851,5,One Little Indian (1973)
836474,5854,4,Slaughterhouse (1987)
836475,5854,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,5,Identification of a Woman (Identificazione di ...
0,9999,5,Terminator 2: Judgment Day (1991)
1,9999,5,"Parent Trap, The (1998)"
2,9999,5,Home Alone 3 (1997)
3,9999,5,Notting Hill (1999)
4,9999,5,"Shawshank Redemption, The (1994)"


### 모델에 활용하기 위한 전처리

사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 user와 artist 각각에 번호를 붙이고 싶습니다. 보통 이런 작업을 __indexing__ 이라고 합니다. 추천시스템, 자연어처리에서 자주 하는 작업들이라 직접 해보시면 좋겠습니다.

In [76]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = movie_df['user_id'].unique() 
title_unique = movie_df['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자 입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [77]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[9999])  
print(title_to_idx['Notting Hill (1999)'])

6039
1151


In [78]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = movie_df['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movie_df):  # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_df['user_id'] = temp_user_data  # data['user_id']을 인덱싱된 Series로 교체해 줍니다.
else:
    print('user_id column indexing Fail!!')
    
# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = movie_df['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(movie_df): 
    print('movie title column indexing OK!!')
    movie_df['title'] = temp_title_data
else:
    print('movie title column indexing Fail!!')

movie_df

user_id column indexing OK!!
movie title column indexing OK!!


,user_id,count,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,92
1,6039,5,1510
2,6039,5,1994
3,6039,5,1151


## 4. CSR matrix 만들기

CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다.

In [79]:
from scipy.sparse import csr_matrix

num_user = movie_df['user_id'].nunique()
num_title = movie_df['title'].nunique()

csr_data = csr_matrix((movie_df['count'], (movie_df.user_id, movie_df.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델 훈련

이전 스텝에서 설명한 Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅시다.
- implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.  
  
  
- 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

In [80]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 __ init __ 파라미터를 살펴보겠습니다.   
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지   
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지  
3. use_gpu : GPU를 사용할 것인지  
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지  

In [81]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=150,
                                   regularization=0.01,
                                   use_gpu=False,
                                   iterations=30,
                                   dtype=np.float32)

In [82]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [83]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

## 6. 훈련된 모델로 나의 선호도 파악하기

__1) 나의 user_id vector와 영화 쇼생크탈출 vector 확인하기__

In [84]:
my_movie, shawshank = user_to_idx[9999], title_to_idx['Shawshank Redemption, The (1994)']
my_movie_vector, shawshank_vector = als_model.user_factors[my_movie], als_model.item_factors[shawshank]

In [85]:
my_movie_vector 

array([ 0.07646539,  0.19678286,  0.4668112 ,  0.01771674, -0.4624646 ,
       -0.27005824, -0.05147791,  0.37931117,  0.21556425,  0.22590415,
        0.12625173,  0.17250834,  0.08730108,  0.52408236,  0.58624244,
       -0.58121544, -0.12258033,  0.6827645 , -0.08162428,  0.09398942,
        0.5677554 , -0.50242156,  0.56153274, -0.0506565 ,  0.14076541,
        0.10345643, -0.80491316, -0.15541768, -0.12489144, -0.7469446 ,
       -0.11469573,  0.29283193, -0.16821657, -0.06832883,  0.01346262,
       -0.2874667 ,  0.3951984 , -0.6640183 , -0.1629786 ,  0.23420894,
       -0.66486496,  0.307177  ,  0.15960506,  0.17545876, -0.5040985 ,
        0.43324846, -0.33254415, -0.03154882, -0.3411111 ,  0.29141507,
       -0.22511403,  0.2808769 ,  0.24617496, -0.30360055,  0.832705  ,
       -0.61186105, -0.37508225,  0.28213355,  0.2150437 ,  0.129221  ,
       -0.32531822, -0.5210914 ,  0.06320646, -0.30957216, -0.3876198 ,
        0.4947483 , -0.13052759,  0.324723  , -0.0875893 , -0.12

In [86]:
shawshank_vector

array([-7.26925675e-03,  9.42128524e-03,  2.43471246e-02,  2.39632539e-02,
        3.28799174e-03,  2.77842544e-02,  1.78680196e-02,  3.92791219e-02,
        9.94162541e-03,  3.00293267e-02,  2.39633638e-02, -1.08793601e-02,
       -7.75725860e-03,  1.83821786e-02,  2.95686908e-02,  2.66481414e-02,
        2.28491686e-02,  2.70922333e-02,  1.48169212e-02,  2.33195983e-02,
        2.73590963e-02, -4.99081751e-03,  3.15200761e-02,  6.99425116e-04,
        1.90930497e-02,  5.51416958e-03, -3.82393412e-02,  1.68704847e-03,
       -1.90827865e-02, -2.33949572e-02,  3.44708338e-02,  1.28628453e-02,
        9.74715222e-03,  1.16339903e-02, -6.35001017e-03,  2.98507791e-03,
        3.19183655e-02, -1.01805758e-02,  1.88919492e-02, -4.58433293e-04,
       -2.22017001e-02,  1.38620650e-02,  6.86435774e-03, -8.26343894e-03,
       -1.06726948e-03,  1.07537238e-02, -4.75754682e-03, -1.31111136e-02,
        3.46360682e-03, -8.15882720e-03,  5.74412523e-03,  2.38924883e-02,
        2.74415314e-02, -

__2) 위의 두 벡터를 곱하면 어떤 값이 나오는지 확인합니다.__

내적 결과가 낮다고 생각되어 factors, iterations를 조금씩 상향 시켜주며 모델 학습을 진행하니 결과가 상승하였다.

In [87]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(my_movie_vector, shawshank_vector)

0.53635305

__3) 학습된 모델이 다른 영화에 대한 선호도를 어떻게 예측할지 다른 영화의 벡터를 가지고 확인해봅니다.__

In [88]:
home_alone = title_to_idx['Home Alone 3 (1997)']
home_alone_vector = als_model.item_factors[home_alone]
np.dot(my_movie_vector, home_alone_vector)

0.08417789

## 7. 비슷한 영화 추천받기

AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 영화를 찾습니다.

In [89]:
favorite_movie = 'Home Alone 3 (1997)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=10)
similar_movie

[(1994, 1.0000001),
 (2363, 0.8318139),
 (1989, 0.82048357),
 (3020, 0.7768847),
 (3022, 0.77367014),
 (2016, 0.77111787),
 (1617, 0.77061623),
 (2015, 0.7658319),
 (1645, 0.76165515),
 (2019, 0.7516407)]

In [90]:
# title_to_idx 를 뒤집어, index로부터 영화 제목을 얻는 dict를 생성합니다.
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Home Alone 3 (1997)',
 'Richie Rich (1994)',
 'Jungle2Jungle (a.k.a. Jungle 2 Jungle) (1997)',
 'Leave It to Beaver (1997)',
 'Problem Child 2 (1991)',
 'Dunston Checks In (1996)',
 'Little Rascals, The (1994)',
 'Home Alone 2: Lost in New York (1992)',
 'Flintstones, The (1994)',
 'Blank Check (1994)']

In [92]:
# get_similar_movies 함수를 통해 다른 영화에 대해서도 확인해볼 수 있다.
def get_similar_movies(movie_title: str):
    movie_id = title_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [93]:
get_similar_movies('Notting Hill (1999)')

['Notting Hill (1999)',
 "My Best Friend's Wedding (1997)",
 "You've Got Mail (1998)",
 'Sleepless in Seattle (1993)',
 'Pretty Woman (1990)',
 'Sabrina (1995)',
 'While You Were Sleeping (1995)',
 'French Kiss (1995)',
 'Four Weddings and a Funeral (1994)',
 'Truth About Cats & Dogs, The (1996)']

In [94]:
get_similar_movies('Shawshank Redemption, The (1994)')

['Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'Pulp Fiction (1994)',
 "Schindler's List (1993)",
 'Fargo (1996)',
 'Good Will Hunting (1997)',
 'GoodFellas (1990)',
 'Dead Man Walking (1995)',
 'Usual Suspects, The (1995)',
 'Amadeus (1984)']

## 8. 내가 좋아할만한 영화 추천 받기

AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받습니다. filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [95]:
user = user_to_idx[9999]

# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(23, 0.40258133),
 (121, 0.36262235),
 (124, 0.33864346),
 (51, 0.30571544),
 (222, 0.2794707),
 (200, 0.25548238),
 (62, 0.24865238),
 (276, 0.24455208),
 (626, 0.21834509),
 (87, 0.20310894),
 (175, 0.19630235),
 (1200, 0.18303409),
 (1178, 0.18043827),
 (340, 0.17802215),
 (233, 0.17759438),
 (666, 0.17576918),
 (336, 0.17439137),
 (248, 0.17235425),
 (269, 0.16672176),
 (384, 0.15967913)]

In [96]:
[idx_to_title[i[0]] for i in movie_recommended]

["Schindler's List (1993)",
 'Silence of the Lambs, The (1991)',
 'Matrix, The (1999)',
 'Fargo (1996)',
 'Pulp Fiction (1994)',
 'Terminator, The (1984)',
 'Total Recall (1990)',
 'North by Northwest (1959)',
 "You've Got Mail (1998)",
 'Braveheart (1995)',
 'Men in Black (1997)',
 'American President, The (1995)',
 'Hoosiers (1986)',
 "My Best Friend's Wedding (1997)",
 'Usual Suspects, The (1995)',
 'Sleepless in Seattle (1993)',
 'Pretty Woman (1990)',
 'Good Will Hunting (1997)',
 'GoodFellas (1990)',
 'Jerry Maguire (1996)']

AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있습니다.

In [97]:
schindelrlist = title_to_idx["Schindler's List (1993)"]
explain = als_model.explain(user, csr_data, itemid=schindelrlist)

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도를 반환합니다.  
어떤 아티스트들이 이 추천에 얼마나 기여하고 있는 걸까요?

In [98]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Shawshank Redemption, The (1994)', 0.336826625681622),
 ('Notting Hill (1999)', 0.05881900447259472),
 ('Parent Trap, The (1998)', 0.017984240736659843),
 ('Home Alone 3 (1997)', 0.005707597928730595),
 ('Terminator 2: Judgment Day (1991)', -0.021735671496217454)]

장르가 가장 유사한 쇼생크 탈출이 가장 큰 기여를 했고 나머지 영화들은 장르가 전혀 달라 큰 기여를 하지 않은 것으로 추측된다.

## 9. 정리

- CSR matrix와 MF모델을 활용하여 비교적 어렵지 않게 간단한 추천시스템을 구현해볼 수 있었다. 추천시스템을 구현하는 건 어렵지 않으나 실제로 사용자의 성향에 맞는 영화를 얼마나 정확하게 추천해주는 문제는 매우 어려운 부분인 것 같다.  
  
  
- 프로젝트에서는 내가 선호하는 영화를 5개 추가하는 것으로 진행했는데 왓챠플레이처럼 더 많은 영화에 대한 선호도 정보를 제공한다면 좀 더 정확한 추천이 이루어질 수 있을 것 같다.  
  
  
- 제공된 데이터의 한계(사용자 id, 영화제목, 평점, 장르) 때문에 더 뛰어난 추천시스템을 만들기 위해서 연령대, 성별 같은 정보도 추가되면 좋을 것 같다.  
  
  
- 유튜브, 음원사이트, 이커머스 등 우리가 활용하고 있는 다양한 앱에서 어떻게 나의 성향을 파악해서 추천 시스템을 제공하는지 궁금했는데 이번 노드를 통해서 약간이나마 그 프로세스를 이해할 수 있어서 유익하였다.